In [1]:
import pandas as pd 
df_KNN = pd.read_hdf('../../data/processed/model_valid/KNN_10.hdf')
df_KNN.head()

2356200    0.9
2356201    0.2
2356202    1.8
2356203    0.1
2356204    1.0
Name: preds, dtype: float64

In [3]:
df_KNN.name = 'knn10_preds'
df_KNN.head()

2356200    0.9
2356201    0.2
2356202    1.8
2356203    0.1
2356204    1.0
Name: knn10_preds, dtype: float64

In [40]:
import os 
data_path = '../../data/processed/model_valid'
def renamed_series(name, path):
    s = pd.read_hdf(os.path.join(data_path, path))
    col_name = s.columns[1]
    s.set_index('ID', inplace=True)
    s = s[col_name]
    s.name = name 
    return s 

In [7]:

train_data = {
    'knn10': 'KNN_10.hdf',
    'knn3': 'KNN_3.hdf',
    'linreg': 'linreg.hdf',
    'xgb_25': 'xgb_25_feat_11_PCA_valid.hdf',
    'xgb_all': 'xgb_all.hdf',
    'true': 'true.hdf'
}

In [8]:
df_train = pd.DataFrame(data={
    name: renamed_series(name, path) for name, path in train_data.items()
})

In [9]:
df_train.head()

,knn10,knn3,linreg,true,xgb_25,xgb_all
2356200,0.9,3.333333,1.314807,1.0,2.890960,2.739996
2356201,0.2,3.333333,0.712276,3.0,2.718085,2.961049
2356202,1.8,0.000000,2.548201,1.0,1.616810,1.201391
2356203,0.1,0.000000,0.000000,0.0,0.089291,0.130663
2356204,1.0,0.000000,1.502367,1.0,1.360490,1.253611


In [24]:
df_train['linreg_error'] = df_train['linreg'] - df_train['true'].clip(0, 20)

In [11]:
df_train.scatter('true', 'linreg_error')
import matplotlib.pyplot as plt
plt.show()

AttributeError: 'DataFrame' object has no attribute 'scatter'

In [26]:
df_train['true_clipped'] = df_train['true'].clip(0, 20)

In [27]:
df_train.head()

,knn10,knn3,linreg,true,xgb_25,xgb_all,linreg_error,true_clipped
2356200,0.9,3.333333,1.314807,1.0,2.890960,2.739996,0.314807,1.0
2356201,0.2,3.333333,0.712276,3.0,2.718085,2.961049,-2.287724,3.0
2356202,1.8,0.000000,2.548201,1.0,1.616810,1.201391,1.548201,1.0
2356203,0.1,0.000000,0.000000,0.0,0.089291,0.130663,0.000000,0.0
2356204,1.0,0.000000,1.502367,1.0,1.360490,1.253611,0.502367,1.0


In [33]:
x_cols = ['knn10', 'knn3', 'linreg', 'xgb_25', 'xgb_all']
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LinearRegression
model = RandomForestClassifier(n_estimators=20, n_jobs=-1)
model = LinearRegression()
preds = cross_val_predict(model, df_train[x_cols], df_train['true_clipped'])

In [54]:
model.fit(df_train[x_cols], df_train['true_clipped'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [34]:
from sklearn.metrics import r2_score, mean_squared_error
from math import sqrt
print('r2_score: ', r2_score(df_train['true_clipped'], preds.clip(0, 20)))
print('mean_squared_error: ', mean_squared_error(df_train['true_clipped'], preds.clip(0, 20)))

r2_score:  0.38441198366041485
mean_squared_error:  0.7488210459690807


In [31]:
def evaluate(preds):
    print('r2_score: ', r2_score(df_train['true_clipped'], preds))
    print('mean_squared_error: ', mean_squared_error(df_train['true_clipped'], preds))
for col in x_cols:
    print(col)
    evaluate(df_train[col])

knn10
r2_score:  0.202110613383488
mean_squared_error:  0.9705782913165268
knn3
r2_score:  -8.091188780521993
mean_squared_error:  11.058814192343604
linreg
r2_score:  0.19391697251358753
mean_squared_error:  0.9805452993862701
xgb_25
r2_score:  0.3609760703133814
mean_squared_error:  0.7773292441145192
xgb_all
r2_score:  0.35850566191878996
mean_squared_error:  0.7803343282761777


In [41]:

test_data = {
    'knn10': 'knn10_test.hdf',
    'knn3': 'KNN_3_test.hdf',
    'linreg': 'linreg_test.hdf',
    'xgb_25': 'xgb_25_feat_11_PCA_test.hdf',
    'xgb_all': 'xgb_all_test.hdf'
}
df_test = pd.DataFrame(data={
    name: renamed_series(name, path) for name, path in test_data.items()
})

In [44]:
df_test.reset_index(inplace=True)

In [64]:
df_test['item_cnt_month'] = model.predict(df_test[x_cols])

In [65]:
df_test[((df_test['item_cnt_month'] > 0.0) & (df_test['item_cnt_month'] < 0.5))].head()

,ID,knn10,knn3,linreg,xgb_25,xgb_all,item_cnt_month
3,21,0.0,0.000000,0.106596,0.292600,0.454868,0.315010
7,62,0.0,0.333333,0.402045,0.308566,0.144193,0.207867
8,69,0.0,0.000000,0.548883,0.095776,0.247778,0.142577
13,114,0.0,0.000000,0.466653,0.228959,0.182046,0.182398
14,193,0.0,0.000000,0.253667,0.281387,0.317101,0.258416


In [62]:
df_test[['ID', 'item_cnt_month']].to_csv

,ID,knn10,knn3,linreg,xgb_25,xgb_all,item_cnt_month
0,0,0.0,0.666667,0.915902,0.889613,0.646006,0.889613
1,2,0.0,0.333333,0.947179,1.633402,1.284492,1.633402
2,15,0.0,0.333333,4.331489,2.740176,2.557366,2.740176
3,21,0.0,0.000000,0.106596,0.292600,0.454868,0.000000
4,23,0.0,0.333333,1.115873,1.172831,0.729194,1.172831


In [ ]:
df+

In [48]:
((df_test['xgb_25'] > 0.0) & (df_test['xgb_25'] < 0.5)).mean()

0.7854948646125117

In [50]:
df_test['item_cnt_month'] = df_test['item_cnt_month'] * (df_test['item_cnt_month'] > 0.5)

In [66]:
df_test[['ID', 'item_cnt_month']].to_csv('../../data/submissions/stacked_model_not_cut.csv', index=False)

In [61]:
df_test2 = df_test
df_test2['item_cnt_month'] = df_test['xgb_25'] * (df_test['xgb_25'] > 0.5)
df_test2[['ID', 'item_cnt_month']].to_csv('../../data/submissions/xgb_25_cut_under_half.csv', index=False)